In [14]:
import pandas as pd
import numpy as np
import pymysql
import open_api2

def dev_db_conn():
    conn = pymysql.connect( 
        host = 호스트, 
        user = 유저, 
        password = 비밀번호, 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor) 
    return conn


def live_db_conn(): 
    conn = pymysql.connect( 
        host = 호스트, 
        user = 유저, 
        password = 비밀번호, 
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor, 
        db = DB명) 
    return conn

In [15]:
with live_db_conn() as conn:
    sql = """
    SELECT T1.iapIsdAcnoEcyVl AS 'TRST_ID',
        T2.category_two AS 'CTGRY', T3.country AS 'NLTY', T3.age AS 'AGE', T3.sexdstn AS 'SEXDSTN',
        CONCAT(LEFT(T1.trDt,4),'-',RIGHT(LEFT(T1.trDt,6),2),'-',RIGHT(T1.trDt,2),' ',LEFT(T1.trTm, 2),':',RIGHT(LEFT(T1.trTm,4),2),':',RIGHT(T1.trTm,2)) AS 'CNSMP_DT',
        T1.trAmt AS 'CNSMP_AMOUNT', COALESCE(SUBSTRING_INDEX(T2.address,' ',1), SUBSTRING_INDEX(T2.address_doro,' ',1)) AS 'Area1',
        COALESCE(SUBSTRING_INDEX(SUBSTRING_INDEX(T2.address,' ',2),' ',-1), SUBSTRING_INDEX(SUBSTRING_INDEX(T2.address,' ',2),' ',-1)) AS 'Area2'
    FROM redtable2021.IA_KTC_TRANSACTION T1
    LEFT JOIN redtable2021.dic_naver_category T2 ON T1.trIsttJngbrNm = T2.name AND T1.trIsttJngbrBzprNo = T2.business_1
    LEFT JOIN (SELECT id, iapIsdAcnoEcyVl, RIGHT(cardNo,4) AS 'right_cardNo', user_id, cardNo, country, age, sexdstn
            FROM redtable2021.IA_KTC_SOURCE
            GROUP BY iapIsdAcnoEcyVl, right_cardNo) T3 ON T1.iapIsdAcnoEcyVl = T3.iapIsdAcnoEcyVl AND RIGHT(T1.iapMskCdno,4) = T3.right_cardNo
    WHERE T1.rndDscd = 2 AND T1.cnctrYn = 'N'
    AND T1.trIsttJngbrNm NOT IN ("계약매장이 아닐때", "- 계약매장이고 0% 페이백으로 되어있는 경우", "페이백X 계약매장이고 5% 페이백으로 되어있는 경우",
        "계약X", "- 계약매장 아닐때 -> 0.2% 페이백", "- 계약매장이고 사업자번호가 2123996827 인 경우",
        "- 계약매장이고 5% 페이백으로 되어있는 경우", "계약매장이고 0% 페이백으로 되어있는 경우", "계약매장이고 5% 페이백으로 되어있는 경우",
        "테스트입니다-ssh-30", "테스트입니다-ssh-0.2", "테스트입니다-일반매장-ssh-0.2",
        "22테스트입니다-일반매장-ssh-0.2", "33테스트입니다-일반매장-ssh-0.2", "44테스트입니다-ssh-0.2",
        "55테스트입니다-ssh-30%", "테스트", "테스트 입금", "테스트 신라", "누적 이벤트2 계약상점O", "테스트 위챗페이 입금")
    ORDER BY T1.id ASC, T1.created_at ASC;
    """
    IA_CARD_TABLE = pd.read_sql(sql, conn)
    
    sql = """
    SELECT T1.userId AS 'TRST_ID',
        T2.category_two AS 'CTGRY', T4.country AS 'NLTY', '' AS 'AGE', '' AS 'SEXDSTN',
      CONCAT(LEFT(T1.approvalDateTime, 4), '-', RIGHT(LEFT(T1.approvalDateTime,6), 2), '-', RIGHT(LEFT(T1.approvalDateTime,8), 2), 
      ' ', LEFT(RIGHT(T1.approvalDateTime,6),2), ':', LEFT(RIGHT(T1.approvalDateTime,4),2), ':', RIGHT(T1.approvalDateTime,2)) AS 'CNSMP_DT',
      T1.trAmount AS 'CNSMP_AMOUNT', COALESCE(SUBSTRING_INDEX(T2.address,' ',1), SUBSTRING_INDEX(T2.address_doro,' ',1)) AS 'Area1',
        COALESCE(SUBSTRING_INDEX(SUBSTRING_INDEX(T2.address,' ',2),' ',-1), SUBSTRING_INDEX(SUBSTRING_INDEX(T2.address,' ',2),' ',-1)) AS 'Area2'
    FROM redtable2021.KONA_TRANSACTION T1
    LEFT JOIN redtable2021.dic_naver_category T2 ON T1.merchantName = T2.name AND T1.bizLicenseNo = T2.business_1
    LEFT JOIN redtable2021.KONA_CARD T3 ON T1.cardNo = T3.card_no
    LEFT JOIN redtable2021.KONA_SOURCE T4 ON T3.id = T4.KONA_CARD_ID
    WHERE T1.trType = '00' AND T1.mti = '0100' AND T1.responseCode = '00' AND T1.id NOT IN (37, 39, 49, 339)
      AND T1.bizTypeName NOT IN ('테스트')
    ORDER BY T1.id ASC, T1.created_at ASC;
    """
    KONA_CARD_TABLE = pd.read_sql(sql, conn)

KONA_CARD_TABLE["CNSMP_AMOUNT"] = KONA_CARD_TABLE["CNSMP_AMOUNT"].apply(int)

# 아이오로라 + 코나 합치기
df = pd.concat([IA_CARD_TABLE, KONA_CARD_TABLE])
df.sort_values(by="CNSMP_DT", inplace=True)
df.reset_index(drop=True, inplace=True)

# ID 임의로 생성하기
df["TRST_CNSMP_ID"] = df.index + 1

df = df[["TRST_CNSMP_ID", "TRST_ID", "CMRCL_AREA_NM", "CTGRY", "NLTY", 
        "AGE", "SEXDSTN", "CNSMP_DT", "CNSMP_AMOUNT", "Area1", "Area2"]]

# 국적 명칭 통일하기
df["NLTY"] = df["NLTY"].replace(to_replace="한국", value="대한민국")
df["NLTY"] = df["NLTY"].replace(to_replace="중화민국", value="중국")

# 국적 미입력 추기입력하기
df.loc[df["TRST_ID"] == "2001203000730880", "NLTY"] = "대한민국"

# 연령 입력값 변경하기
df.loc[df["AGE"] == "0대", "AGE"] = "A00" # 유아 및 어린이
df.loc[df["AGE"] == "10대", "AGE"] = "A10" # 청소년
df.loc[df["AGE"] == "20대", "AGE"] = "A20" # 청년
df.loc[df["AGE"] == "30대", "AGE"] = "A30" # 성인 초기
df.loc[df["AGE"] == "40대", "AGE"] = "A40" # 성인 중반
df.loc[df["AGE"] == "50대", "AGE"] = "A50" # 성인 후반
df.loc[df["AGE"] == "60대", "AGE"] = "A60" # 장년 및 노년
df.loc[df["AGE"].isnull() == True, "AGE"] = "PNTS" # 연령대를 밝히고 싶지 않은 경우
df.loc[df["AGE"] == "", "AGE"] = "PNTS" # 연령대를 밝히고 싶지 않은 경우

# 성별 입력값 변경하기
df.loc[df["SEXDSTN"] == "남성", "SEXDSTN"] = "M" # 전통적으로 남성으로 식별되는 성별
df.loc[df["SEXDSTN"] == "여성", "SEXDSTN"] = "F" # 전통적으로 여성으로 식별되는 성별
df.loc[df["SEXDSTN"] == "트랜스여성", "SEXDSTN"] = "TF" # 지정 성별이 남성이지만 여성으로 정체화하는 사람
df.loc[df["SEXDSTN"] == "트랜스남성", "SEXDSTN"] = "TM" # 지정 성별이 여성이지만 남성으로 정체화하는 사람
df.loc[df["SEXDSTN"].isnull() == True, "SEXDSTN"] = "PNTS" # 성별을 밝히고 싶지 않는 경우
df.loc[df["SEXDSTN"] == "", "SEXDSTN"] = "PNTS" # 성별을 밝히고 싶지 않는 경우

In [16]:
df

,TRST_CNSMP_ID,TRST_ID,CTGRY,NLTY,AGE,SEXDSTN,CNSMP_DT,CNSMP_AMOUNT,Area1,Area2
0,1,2001203000728900,편의점,대한민국,PNTS,PNTS,2023-07-13 08:25:43,2600,서울특별시,중구
1,2,2001203000730880,"카페,디저트",대한민국,PNTS,PNTS,2023-07-13 13:19:35,5400,서울특별시,중구
2,3,2001203000731485,"카페,디저트",대한민국,A20,M,2023-07-13 14:48:14,10500,서울특별시,중구
3,4,2001203000731427,"카페,디저트",대한민국,PNTS,PNTS,2023-07-13 14:49:01,10600,서울특별시,중구
4,5,2001203000731485,"카페,디저트",대한민국,A20,M,2023-07-13 14:51:24,13100,서울특별시,중구
...,...,...,...,...,...,...,...,...,...,...
62013,62014,3101858786,편의점,중국,PNTS,PNTS,2024-08-14 01:49:12,5700,부산광역시,수영구
62014,62015,2001204000366071,편의점,중국,PNTS,PNTS,2024-08-14 03:27:24,2000,서울특별시,영등포구
62015,62016,3101858786,편의점,중국,PNTS,PNTS,2024-08-14 04:38:35,3550,부산광역시,수영구
62016,62017,2001204000366071,카페,중국,PNTS,PNTS,2024-08-14 07:10:07,23000,서울특별시,영등포구


In [17]:
# 상권 데이터 추출 후 매핑하기
with live_db_conn() as conn:
    sql = """
    SELECT CONCAT(T2.name2, ' ', T1.name) AS 'CMRCL_AREA_NM'
    FROM redtable2021.commercial_area T1
    INNER JOIN redtable2021.location_master T2 ON T1.location_master_id = T2.id
    WHERE T1.`status` = 'normal'
    """
    df_commercial = pd.read_sql(sql, conn)
    
result1, result2 = [], []

for i in range(len(df_commercial["CMRCL_AREA_NM"])):
    standard_text = open_api2.preprocess(df_commercial["CMRCL_AREA_NM"][i])
    result1.append(standard_text[0])
    result2.append(standard_text[1])
    
df_commercial["Area1"] = result1
df_commercial["Area2"] = result2

In [18]:
df_commercial

,CMRCL_AREA_NM,Area1,Area2
0,부산 중구,부산광역시,중구
1,부산 서구,부산광역시,서구
2,부산 동구,부산광역시,동구
3,부산 영도,부산광역시,영도구
4,부산 부산진구,부산광역시,부산진구
...,...,...,...
256,전북 전체,None,None
257,전남 전체,None,None
258,경북 전체,None,None
259,경남 전체,None,None


In [19]:
df_final = pd.merge(df, df_commercial, on=["Area1", "Area2"], how='left')
df_final = df_final[['TRST_CNSMP_ID', 'TRST_ID', 'CMRCL_AREA_NM', 'CTGRY', 'NLTY', 
                     'AGE', 'SEXDSTN', 'CNSMP_DT', 'CNSMP_AMOUNT', 'Area1', 'Area2']]
df_final = df_final.replace({np.nan:None, '':None})

In [20]:
df_final

,TRST_CNSMP_ID,TRST_ID,CMRCL_AREA_NM,CTGRY,NLTY,AGE,SEXDSTN,CNSMP_DT,CNSMP_AMOUNT,Area1,Area2
0,1,2001203000728900,서울 중구,편의점,대한민국,PNTS,PNTS,2023-07-13 08:25:43,2600,서울특별시,중구
1,1,2001203000728900,서울 명동,편의점,대한민국,PNTS,PNTS,2023-07-13 08:25:43,2600,서울특별시,중구
2,2,2001203000730880,서울 중구,"카페,디저트",대한민국,PNTS,PNTS,2023-07-13 13:19:35,5400,서울특별시,중구
3,2,2001203000730880,서울 명동,"카페,디저트",대한민국,PNTS,PNTS,2023-07-13 13:19:35,5400,서울특별시,중구
4,3,2001203000731485,서울 중구,"카페,디저트",대한민국,A20,M,2023-07-13 14:48:14,10500,서울특별시,중구
...,...,...,...,...,...,...,...,...,...,...,...
95611,62016,3101858786,부산 수영,편의점,중국,PNTS,PNTS,2024-08-14 04:38:35,3550,부산광역시,수영구
95612,62017,2001204000366071,서울 영등포,카페,중국,PNTS,PNTS,2024-08-14 07:10:07,23000,서울특별시,영등포구
95613,62017,2001204000366071,서울 여의도,카페,중국,PNTS,PNTS,2024-08-14 07:10:07,23000,서울특별시,영등포구
95614,62018,2001204000386842,서울 영등포,카페,대한민국,PNTS,PNTS,2024-08-14 08:22:49,3000,서울특별시,영등포구


In [28]:
with dev_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'TRST_CNSMP_TRND'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [34]:
with dev_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        TRUNCATE TABLE DATA_VOUCHER.TRST_CNSMP_TRND;
        """
        curs.execute(sql)
        
        sql = f"""
        INSERT IGNORE INTO DATA_VOUCHER.TRST_CNSMP_TRND({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df_final.apply(lambda row:(row["TRST_CNSMP_ID"], row["TRST_ID"], row["CMRCL_AREA_NM"], row["CTGRY"],
                                  row["NLTY"], row["AGE"], row["SEXDSTN"], row["CNSMP_DT"],
                                  row["CNSMP_AMOUNT"]), axis=1).tolist()
        curs.executemany(sql, val)